In [3]:
import os, socket, subprocess, sys, time, shlex

PORT = int(os.environ.get("SHINY_PORT", "8787"))
APP_PATH = os.environ.get("SHINY_APP", "app.R")

def port_open(port:int) -> bool:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(0.2)
    try:
        s.connect(("127.0.0.1", port))
        return True
    except Exception:
        return False
    finally:
        s.close()

if not port_open(PORT):
    # Launch R Shiny in the background using Rscript
    cmd = f'Rscript -e "shiny::runApp(\\"{APP_PATH}\\", host=\\"0.0.0.0\\", port={PORT}, launch.browser=FALSE)"'
    # Use Popen so Voilà can keep running
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, preexec_fn=os.setpgrp if hasattr(os, "setpgrp") else None)
    # Give it a brief moment to bind
    for _ in range(20):
        if port_open(PORT):
            break
        time.sleep(0.2)

# Show where we think the app is listening (inside the pod)
print(f"Shiny should be listening on 0.0.0.0:{PORT}")

Shiny should be listening on 0.0.0.0:8787


In [4]:
import os
from ipywidgets import Button, VBox, HTML
from IPython.display import display

# PORT = int(os.environ.get("SHINY_PORT", "8787"))

base = os.environ.get("JUPYTERHUB_SERVICE_PREFIX", "/")
dynamic_url = f"{base}proxy/{PORT}/"

# Redirect your users
hardcoded_url = f"https://jupyterhub.haskoning.app/user/user-redirect/proxy/{PORT}/"

title = HTML("<h2>Shiny App Launcher</h2><p>Click to open in a new tab.</p>")
button_html = HTML(f'''
  <a href="{dynamic_url}" target="_blank" style="
      display:inline-block;
      padding:12px 18px;
      border-radius:10px;
      text-decoration:none;
      border:1px solid #ccc;
      font-weight:600;
  ">🚀 Open Shiny (dynamic)</a>
''')

display(VBox([title, button_html]))